In [1]:
!pip install yt-dlp
!pip install nltk

  Using cached yt_dlp-2025.11.12-py3-none-any.whl.metadata (180 kB)
Using cached yt_dlp-2025.11.12-py3-none-any.whl (3.3 MB)

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached regex-2025.11.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
Using cached regex-2025.11.3-cp313-cp313-macosx_11_0_arm64.whl (288 kB)
Using cached click-8.3.1-py3-none-any.whl (108 kB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [nltk]4/5 [nltk]b]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip inst

In [17]:
!pip install -U yt-dlp certifi


  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
Using cached certifi-2025.11.12-py3-none-any.whl (159 kB)
  Attempting uninstall: certifi
    Found existing installation: certifi 2025.8.3
    Uninstalling certifi-2025.8.3:
      Successfully uninstalled certifi-2025.8.3


### Step 1: YouTube Subtitle Extraction

Using `yt-dlp` Python module: it can download **auto-generated** or **uploaded subtitles** directly in .vtt or .srt format without downloading the entire video.

In [4]:
import yt_dlp
import os
import json
import re
import glob
import unicodedata

VIDEO_URLS = [
    "https://www.youtube.com/watch?v=dMfIh3Q_Rgc",
    "https://www.youtube.com/watch?v=Mdpf0n-tGvE",
    "https://www.youtube.com/watch?v=gdiao7L9GjE",
    "https://www.youtube.com/watch?v=6KU054gmpfg"
]

OUTPUT_FOLDER_DATA = "../data"
os.makedirs(OUTPUT_FOLDER_DATA, exist_ok=True)
OUTPUT_FOLDER_RAW = "../data/raw"
os.makedirs(OUTPUT_FOLDER_RAW, exist_ok=True)
OUTPUT_FOLDER_METADATA = "../data/metadata"
os.makedirs(OUTPUT_FOLDER_METADATA, exist_ok=True)
METADATA_FILE = "../data/metadata/subtitle_metadata.json"
metadata_list = []

def sanitize_filename(name):
    # Normalize unicode (full-width → half-width)
    name = unicodedata.normalize('NFKC', name)
    # # Remove invalid filesystem characters
    # name = re.sub(r'[\\/*?:"<>|]', "", name)
    # # Remove full-width colon and other unwanted punctuation
    # name = re.sub(r'[：]', " ", name)
    # # Collapse multiple spaces
    # name = re.sub(r'\s+', ' ', name)
    # return name.strip()
    # Remove invalid filesystem characters and punctuation that may break glob
    name = re.sub(r'[\\/*?:"<>|]', "", name)
    # Replace remaining special punctuation with a space
    name = re.sub(r'[：]', ":", name)  # full-width colon → normal colon
    return name.strip()

for url in VIDEO_URLS:
    # Get video info
    with yt_dlp.YoutubeDL({'skip_download': True}) as ydl_info:
        info_dict = ydl_info.extract_info(url, download=False)
        title = sanitize_filename(info_dict.get('title', 'unknown_title'))

    # Download subtitles
    ydl_opts = {
        'skip_download': True,
        'writesubtitles': True,
        'writeautomaticsub': True,
        'subtitlesformat': 'vtt',
        'subtitleslangs': ['en'],
        'outtmpl': os.path.join(OUTPUT_FOLDER_RAW, '%(title)s.%(ext)s')
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # Find the actual downloaded file
    all_vtt_files = glob.glob(os.path.join(OUTPUT_FOLDER_RAW, "*.vtt"))
    matched_file = None
    for f in all_vtt_files:
        fname = os.path.basename(f)
        # Remove language code and extension for comparison
        fname_no_ext = re.sub(r'\.en\.vtt$', '', fname, flags=re.IGNORECASE)
        fname_no_ext = re.sub(r'\.vtt$', '', fname_no_ext, flags=re.IGNORECASE)
        fname_no_ext = unicodedata.normalize('NFKC', fname_no_ext)
        fname_no_ext = sanitize_filename(fname_no_ext)
        if title == fname_no_ext:
            matched_file = f
            break

    if matched_file:
        final_file = os.path.join(OUTPUT_FOLDER_RAW, f"{title}.vtt")
        if matched_file != final_file:
            os.rename(matched_file, final_file)
    else:
        print(f"[WARNING] Subtitle not found for {title}")
        final_file = None

    metadata_list.append({
        "video_title": title,
        "url": url,
        "subtitle_file": final_file
    })

# Save metadata
with open(METADATA_FILE, "w", encoding="utf-8") as f:
    json.dump(metadata_list, f, indent=4, ensure_ascii=False)

print("✅ Subtitles downloaded and metadata saved!")


[youtube] Extracting URL: https://www.youtube.com/watch?v=dMfIh3Q_Rgc
[youtube] dMfIh3Q_Rgc: Downloading webpage


[youtube] dMfIh3Q_Rgc: Downloading android sdkless player API JSON
[youtube] dMfIh3Q_Rgc: Downloading web safari player API JSON


[youtube] dMfIh3Q_Rgc: Downloading m3u8 information


[youtube] Extracting URL: https://www.youtube.com/watch?v=dMfIh3Q_Rgc
[youtube] dMfIh3Q_Rgc: Downloading webpage


[youtube] dMfIh3Q_Rgc: Downloading android sdkless player API JSON
[youtube] dMfIh3Q_Rgc: Downloading web safari player API JSON


...

[youtube] 6KU054gmpfg: Downloading android sdkless player API JSON
[youtube] 6KU054gmpfg: Downloading web safari player API JSON


[youtube] 6KU054gmpfg: Downloading m3u8 information


[info] 6KU054gmpfg: Downloading subtitles: en


[info] 6KU054gmpfg: Downloading 1 format(s): 96
[info] Writing video subtitles to: ../data/raw/How to build a Web3 game on the Base blockchain.en.vtt


[download] Destination: ../data/raw/How to build a Web3 game on the Base blockchain.en.vtt
[download] 100% of   82.14KiB in 00:00:00 at 503.40KiB/s
✅ Subtitles downloaded and metadata saved!


### Step 2: Preprocess the Subtitles

Goal: Clean and prepare the subtitle text so it can be effectively used for embedding and question-answering in the RAG system.

Files Format: They are in WEBVTT format with both timestamps and HTML-like <c> tags for word-level timing, plus duplicate lines of text.

Dublicate text -> Why? : YouTube repeats each segment in both word-level timing and sentence-level captions. ->

How to:

1. Removed all timestamps (00:00:00.160 --> 00:00:03.270) and metadata (align:start position:0%).

2. Removed all <c> tags and word-level fragments.

3. Removed duplicate consecutive lines — so each sentence only appears once.

4. Joined all lines into a single readable paragraph.

In [5]:
import re
import os
import json

RAW_FOLDER = "../data/raw"
PROCESSED_FOLDER = "../data/processed"
os.makedirs(PROCESSED_FOLDER, exist_ok=True)

def clean_vtt_to_json(file_path, output_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    subtitles = []
    prev_text = ""

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Skip headers
        if line.startswith("WEBVTT") or line.startswith("Kind:") or line.startswith("Language:"):
            i += 1
            continue

        # Timestamp line
        if "-->" in line:
            # Extract clean start and end timestamps
            match = re.match(r'(\d{2}:\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}:\d{2}\.\d{3})', line)
            if match:
                start_time = match.group(1)
                end_time = match.group(2)

                # Collect the subtitle text (may span multiple lines)
                text_lines = []
                i += 1
                while i < len(lines) and lines[i].strip() != "" and "-->" not in lines[i]:
                    # Remove HTML-like tags
                    cleaned_line = re.sub(r'<.*?>', '', lines[i].strip())
                    if cleaned_line and cleaned_line != prev_text:  # remove duplicates
                        text_lines.append(cleaned_line)
                        prev_text = cleaned_line
                    i += 1

                if text_lines:
                    subtitle_text = ' '.join(text_lines)
                    subtitles.append({
                        "start_time": start_time,
                        "end_time": end_time,
                        "text": subtitle_text
                    })
        else:
            i += 1

    # Save cleaned subtitles as JSON
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(subtitles, f, ensure_ascii=False, indent=2)

# Process all .vtt files
for file_name in os.listdir(RAW_FOLDER):
    if file_name.endswith(".vtt"):
        input_file = os.path.join(RAW_FOLDER, file_name)
        output_file = os.path.join(PROCESSED_FOLDER, file_name.replace(".vtt", ".json"))
        clean_vtt_to_json(input_file, output_file)

print("All subtitles cleaned and saved as JSON!")


All subtitles cleaned and saved as JSON!


### Step 3: Chunk the Text with real timestamps

Goal: Split the cleaned text into **chunks* so the embeddings capture context without exceeding token limits.

- Size: 500–1000 tokens per chunk (~3–5 paragraphs).

Why chunking matters

- Embeddings work better when text segments are cohesive and not too long.

- Too short → context is lost, more calls to the vector DB.

- Too long → embedding might truncate important info.

```json
{
  "video_title": "Blockchain Game Development 101",
  "url": "https://youtube.com/xxxx",
  "text_chunk": "In this part of the video, we discuss why blockchain gaming is becoming popular...",
  "start_time": "00:00",
  "end_time": "02:30"
}

In [6]:
import json
import os
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

METADATA_FILE = "../data/metadata/subtitle_metadata.json"
PROCESSED_FOLDER = "../data/processed"
OUTPUT_FOLDER = "../output"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
OUTPUT_DATASET = os.path.join(OUTPUT_FOLDER, "rag_dataset.json")

MAX_TOKENS = 300  # number of words per chunk

# Load metadata from Step 1
with open(METADATA_FILE, "r", encoding="utf-8") as f:
    metadata_list = json.load(f)

# Map base filename to URL
file_to_url = {}
for meta in metadata_list:
    base_name = os.path.splitext(os.path.basename(meta["subtitle_file"]))[0]
    file_to_url[base_name] = meta["url"]

final_dataset = []

for json_file in os.listdir(PROCESSED_FOLDER):
    if not json_file.endswith(".json"):
        continue

    video_json_path = os.path.join(PROCESSED_FOLDER, json_file)

    # Load cleaned subtitles from Step 2
    with open(video_json_path, "r", encoding="utf-8") as f:
        subtitles = json.load(f)

    # Match filename to metadata
    video_title = os.path.splitext(json_file)[0]
    url = file_to_url.get(video_title, "URL_NOT_FOUND")

    # Chunking
    current_chunk = []
    current_start = None
    current_end = None
    word_count = 0

    for entry in subtitles:
        words = entry["text"].split()
        if current_start is None:
            current_start = entry["start_time"]
        current_end = entry["end_time"]

        current_chunk.extend(words)
        word_count += len(words)

        if word_count >= MAX_TOKENS:
            final_dataset.append({
                "video_title": video_title,
                "url": url,
                "text_chunk": " ".join(current_chunk),
                "start_time": current_start,
                "end_time": current_end
            })
            # Reset for next chunk
            current_chunk = []
            current_start = None
            current_end = None
            word_count = 0

    # Save remaining chunk
    if current_chunk:
        final_dataset.append({
            "video_title": video_title,
            "url": url,
            "text_chunk": " ".join(current_chunk),
            "start_time": current_start,
            "end_time": current_end
        })

# Save final RAG dataset
with open(OUTPUT_DATASET, "w", encoding="utf-8") as f:
    json.dump(final_dataset, f, indent=2, ensure_ascii=False)

print(f"✅ Created dataset with {len(final_dataset)} chunks, all with correct URLs.")


✅ Created dataset with 34 chunks, all with correct URLs.


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sofiazogkza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Issues** I run into so far:

Step 2 cleaning – at first, I cleaned the subtitles but ended up stripping all the timestamps. The text looked nice and clean, but it had no timing info, which is kind of useless if you want to link chunks back to the video.

Step 3 chunking – when I tried to chunk that cleaned text, things got messy. Sentences were repeating, chunks were huge and awkward. Basically, it was a big jumble of text with no real structure.

File mismatch – Step 3 expected .vtt files, but Step 2 was outputting JSON. So I had to tweak Step 3 to read the processed JSON files instead of raw VTT. That way, everything lined up: cleaned text + timestamps + proper structure.

Chunk size – I wasn’t sure how big each chunk should be. I decided ~100 tokens per chunk works fine, enough context for the model without being too big. You could go bigger if you want more context, but this is a good balance.

# 🧪 TESTING - Last step

In [7]:
import json
import random

with open(OUTPUT_DATASET, "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Randomly sample 5 chunks
samples = random.sample(dataset, 5)

for s in samples:
    print(f"--- {s['video_title']} ---")
    print(f"URL: {s['url']}")
    print(f"Start: {s['start_time']} | End: {s['end_time']}")
    print(f"Text: {s['text_chunk'][:200]}...")  # preview first 200 chars
    print("\n")
    
# Optional: Check for empty or duplicate chunks
empty_chunks = [d for d in dataset if not d['text_chunk'].strip()]
duplicates = len(dataset) - len({d['text_chunk'] for d in dataset})

print(f"Empty chunks: {len(empty_chunks)}")
print(f"Duplicate chunks: {duplicates}")


--- How to Build a Blockchain Game ---
URL: https://www.youtube.com/watch?v=Mdpf0n-tGvE
Start: 00:28:30.000 | End: 00:30:47.830
Text: this should not burn my kitten and now have two grumpy cats so i don't show actually how many i have here but that's another thing i could easily do using another hook use balance this will give me ho...


--- How to Build a Blockchain Game ---
URL: https://www.youtube.com/watch?v=Mdpf0n-tGvE
Start: 00:30:47.840 | End: 00:31:17.080
Text: and use the web3 button to call things on it and that's how in less than a few hours you can build any web 3 app that you can possibly imagine that is the magic of the web and we absolutely love this ...


--- How to Build a Blockchain Game ---
URL: https://www.youtube.com/watch?v=Mdpf0n-tGvE
Start: 00:24:56.720 | End: 00:26:44.149
Text: all the read functions here and it shows you some nice capabilities here the switch to javascript and writing data i'm interested in claim so here actually it gives me the exact syntax t